In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation
import glob

def get_projection_matrix(R, C, K):
    return np.dot(K, np.hstack((R, -np.dot(R, C.reshape(3, 1)))))


def h_transform(pts):
    return np.hstack((pts, np.ones((pts.shape[0], 1))))

def get_rotation(Q, type_ = 'q'):
    if type_ == 'q':
        R = Rotation.from_quat(Q)
        return R.as_matrix()
    elif type_ == 'e':
        R = Rotation.from_rotvec(Q)
        return R.as_matrix()

def get_quaternion(R2):
    Q = Rotation.from_matrix(R2)
    return Q.as_quat()

def get_euler(R2):
    euler = Rotation.from_matrix(R2)
    return euler.as_rotvec()

def image_count(data_path):
    image_count = len(glob.glob(data_path + "*.png"))

    return image_count

def camera_matrix(data_path):

    filename = data_path + 'calibration.txt'
    # Initialize a 3x3 matrix
    matrix = np.zeros((3, 3))

    # Read the contents of the file and fill the matrix
    with open(filename, 'r') as f:
        for i, line in enumerate(f):
            values = line.strip().split()
            for j, value in enumerate(values):
                matrix[i][j] = float(value)

    # Return the matrix
    return matrix


def extract_features(data_path):
    num_of_imgs = image_count(data_path)

    feature_rgb = []
    feature_x = []
    feature_y = []
    feature_flag = []

    for i in range(1, num_of_imgs):
        matching_fname = data_path + "matching" + str(i) + ".txt"
        fname = open(matching_fname, "r")
        nFeatures = 0
        for j, row in enumerate(fname):
            if j == 0:
                row_elems = row.split(':')
                nFeatures = int(row_elems[1])
            else:
                row_x = np.zeros((1, num_of_imgs))
                row_y = np.zeros((1, num_of_imgs))
                row_flag = np.zeros((1, num_of_imgs), dtype=int)
                row_elems = row.split()
                features = [float(x) for x in row_elems]
                features = np.array(features)

                num_of_matches = features[0]
                red = features[1]
                green = features[2]
                blue = features[3]

                feature_rgb.append([red, green, blue])

                src_x = features[4]
                src_y = features[5]

                row_x[0, i-1] = src_x
                row_y[0, i-1] = src_y
                row_flag[0, i-1] = 1

                j = 1
                while num_of_matches > 1:
                    img_id = int(features[5+j])
                    img_id_x = features[6+j]
                    img_id_y = features[7+j]
                    j = j+3
                    num_of_matches = num_of_matches - 1

                    row_x[0, img_id - 1] = img_id_x
                    row_y[0, img_id - 1] = img_id_y
                    row_flag[0, img_id - 1] = 1

                feature_x.append(row_x)
                feature_y.append(row_y)
                feature_flag.append(row_flag)

    return np.array(feature_x).reshape(-1, num_of_imgs), \
        np.array(feature_y).reshape(-1, num_of_imgs), np.array(feature_flag).reshape(-1, num_of_imgs), \
            np.array(feature_rgb).reshape(-1, 3)


def reprojection_error(X, p1, p2, R1, C1, R2, C2, K):
    P1 = get_projection_matrix(R1, C1, K)
    P2 = get_projection_matrix(R2, C2, K)

    p1_1t, p1_2t, p1_3t = P1
    p1_1t, p1_2t, p1_3t = p1_1t.reshape(1,4), p1_2t.reshape(1,4), p1_3t.reshape(1,4)
    p2_1t, p2_2t, p2_3t = P2
    p2_1t, p2_2t, p2_3t = p2_1t.reshape(1,4), p2_2t.reshape(1,4), p2_3t.reshape(1,4)

    X = X.reshape(4,1)

    # Reprojection error w.r.t camera 1 ref
    u1, v1 = p1[0], p1[1]

    u1_proj = np.divide(p1_1t.dot(X), p1_3t.dot(X))
    v1_proj = np.divide(p1_2t.dot(X), p1_3t.dot(X))

    error1 =  np.square(v1 - v1_proj) + np.square(u1 - u1_proj)

    # Reprojection error w.r.t camera 2 ref
    u2, v2 = p2[0], p2[1]

    u2_proj = np.divide(p2_1t.dot(X), p2_3t.dot(X))
    v2_proj = np.divide(p2_2t.dot(X), p2_3t.dot(X))

    error2 = np.square(v2 - v2_proj) + np.square(u2 - u2_proj)

    return error1, error2


In [ ]:
import numpy as np
import cv2

def get_fundamental_matrix(pts1, pts2):
    # Normalize the points to improve numerical stability
    pts1_norm = cv2.normalize(pts1, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    pts2_norm = cv2.normalize(pts2, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
    # Build the A matrix
    A = np.zeros((len(pts1), 9))
    for i in range(len(pts1)):
        x1, y1 = pts1_norm[i]
        x2, y2 = pts2_norm[i]
        A[i] = [x2*x1, x2*y1, x2, y2*x1, y2*y1, y2, x1, y1, 1]
    
    # Find the singular value decomposition of A
    _, _, v = np.linalg.svd(A)
    
    # The fundamental matrix is the last column of V
    F = v[-1].reshape(3, 3)
    
    # Enforce rank-2 constraint by performing SVD and setting smallest singular value to 0
    u, s, vh = np.linalg.svd(F)
    s[-1] = 0
    S = np.diag(s)
    F = np.dot(np.dot(u, S), vh)
    
    # Denormalize the fundamental matrix
    T = np.array([[1/np.amax(pts1[:,0]), 0, 0], 
                  [0, 1/np.amax(pts1[:,1]), 0], 
                  [0, 0, 1]])
    F = np.dot(np.dot(T.T, F), T)
    
    return F


def get_epipoles(F):
    # Find the left and right singular vectors of F
    _, _, v = np.linalg.svd(F)
    e1 = v[-1]
    
    _, _, v = np.linalg.svd(F.T)
    e2 = v[-1]
    
    # Normalize the epipoles
    e1 /= e1[2]
    e2 /= e2[2]
    
    return e1, e2

def epipolars_line(F, pts1, pts2):
    # Compute epipolar line in image 2 from v1
    l2 = np.dot(F, pts1)
    l1 = np.dot(F.T, pts2)

     # Normalize the line
    l2_norm = np.sqrt(l2[0]**2 + l2[1]**2)
    l2 /= l2_norm

    l1_norm = np.sqrt(l1[0]**2 + l1[1]**2)
    l1 /= l1_norm

    return l1, l2


In [ ]:
import numpy as np
from numpy.linalg import svd
import random

random.seed(19)

"""
def find_fundamental_matrix(x1, x2):
    # Normalize the points to improve the stability of the computation
    x1_norm = (x1 - np.mean(x1, axis=0)) / np.std(x1, axis=0)
    x2_norm = (x2 - np.mean(x2, axis=0)) / np.std(x2, axis=0)

    # Construct the matrix A for the fundamental matrix
    A = np.hstack((x1_norm, np.ones((x1.shape[0], 1))))
    B = np.hstack((x2_norm, np.ones((x2.shape[0], 1))))
    AB = np.einsum('ij,ik->ijk', A, B)
    AB = AB.reshape((-1, AB.shape[-1]))

    # Solve for the fundamental matrix using SVD
    _, _, V = svd(AB)
    F = V[-1, :].reshape(3, 3)

    # Enforce the rank-2 constraint on the fundamental matrix
    U, S, V = svd(F)
    S[-1] = 0
    F = np.dot(U, np.dot(np.diag(S), V))

    # Denormalize the fundamental matrix
    F = np.dot(np.dot(np.transpose(x2), F), x1)

    return F

"""

def compute_residuals(F, x1, x2):
    # Compute the epipolar line for each point in x1
    l = np.dot(F, np.vstack((x1.T, np.ones((1, x1.shape[0])))))
    d = np.sqrt(l[0,:]**2 + l[1,:]**2)
    l = l / d

    # Compute the distance from each point in x2 to its corresponding epipolar line
    x2_hom = np.vstack((x2.T, np.ones((1, x2.shape[0]))))
    distances = np.abs(np.sum(l * x2_hom, axis=0))

    return distances

def find_inliers(x1, x2, F, threshold):
    # Compute the residuals for each correspondence
    residuals = compute_residuals(F, x1, x2)

    # Identify the inliers based on the residuals
    inliers = residuals < threshold

    return inliers

def ransac_fundamental_matrix(x1, x2, num_iterations, threshold):
    best_F = None
    best_num_inliers = 0
    for i in range(num_iterations):
        # Randomly select 8 correspondences
        idx = np.random.choice(x1.shape[0], 8, replace=False)
        F = get_fundamental_matrix(x1[idx, :], x2[idx, :])

        # Compute the number of inliers
        inliers = find_inliers(x1, x2, F, threshold)
        num_inliers = np.sum(inliers)

        # Update the best estimate if necessary
        if num_inliers > best_num_inliers:
            best_F = F
            best_num_inliers = num_inliers

    # Refit the model using all of the inliers
    inliers = find_inliers(x1, x2, best_F, threshold)
    best_F = get_fundamental_matrix(x1[inliers, :], x2[inliers, :])

    return best_F, inliers


In [ ]:
def getEssentialMatrix(K, F):
    E = K.T.dot(F).dot(K)
    U,s,V = np.linalg.svd(E)
    s = [1,1,0]
    E_corrected = np.dot(U,np.dot(np.diag(s),V))
    return E_corrected

In [ ]:
import numpy as np
from scipy.linalg import svd

def decompose_essential_matrix(E):
    """
    Decompose the essential matrix into the relative rotation and translation between two cameras.

    Args:
        E (numpy.ndarray): Essential matrix (3x3).

        K (numpy.ndarray): Camera matrix (3x3).

    Returns:
        numpy.ndarray: Relative rotation (3x3).
        numpy.ndarray: Camera Centers.
    """

    # SVD of the essential matrix
    U, S, Vt = svd(E)

    R = []
    C = []

    
    C1 = U[:, 2]
    C.append(C1)
    C2 = -U[:2]
    C.append(C2)
    C3 = U[:2]
    C.append(C3)
    C4 = -U[:2]
    C.append(C4)
    
    # Check the determinant of U and Vt
    if np.linalg.det(U) < 0:
        U *= -1
    if np.linalg.det(Vt) < 0:
        Vt *= -1

    # Define the matrices for the two possible solutions
    W = np.array([[0, -1, 0], [1, 0, 0], [0, 0, 1]])

    R1 = U @ W @ Vt
    R.append(R1)

    R2 = U @ W @ Vt
    R.append(R2)

    R3 = U @ W.T @ Vt
    R.append(R3)

    R4 = U @ W.T @ Vt
    R.append(R4)


    for i in range(4):
        if (np.linalg.det(R[i]) < 0):
            R[i] = -R[i]
            C[i] = -C[i]
    return R, C


In [ ]:
import numpy as np

# Given the rotation matrix R and camera pose t, and the 3D points P1 and P2 in homogeneous coordinates,
# check if the points satisfy the Triangulation Cheirality Condition
def triangulate(K, R1, C1, R2, C2, x1, x2):
   
    # Compute the projection matrices P1 and P2
    P1 = np.dot(K, np.dot(R1, np.hstack((np.eye(3), -C1.reshape(3, 1)))))
    P2 = np.dot(K, (R2, np.hstack((np.eye(3), -C2.reshape(3, 1)))))

    # Construct the A matrix
    A = np.vstack((x1[0] * P1[2] - P1[0],
                   x1[1] * P1[2] - P1[1],
                   x2[0] * P2[2] - P2[0],
                   x2[1] * P2[2] - P2[1]))




    # Compute the SVD of the A matrix
    _, _, V = np.linalg.svd(A)

    # Extract the 3D point from the last column of V
    X = V[-1, :4]

    # Normalize the homogeneous coordinate
    X /= X[3]

    
    return X[:3]


In [ ]:
import numpy as np

def cheirality_check(C_list, R_list, X_list):
    # initialize a list to store valid camera poses and their corresponding 3D points
    valid_poses = []
    valid_X = []

    # loop over all combinations of two cameras
    for i in range(4):
        for j in range(i+1, 4):
            # calculate the camera matrix for each pair of cameras
            P1 = np.dot(R_list[i], np.hstack((np.identity(3), -C_list[i].reshape(3, 1))))
            P2 = np.dot(R_list[j], np.hstack((np.identity(3), -C_list[j].reshape(3, 1))))

            # check cheirality condition for each triangulated point
            num_valid = 0
            X_valid = []
            for k in range(len(X_list)):
                X_hom = np.hstack((X_list[k], 1))
                x1_hom = np.dot(P1, X_hom)
                x2_hom = np.dot(P2, X_hom)

                if (X_hom[2] > 0) and (x1_hom[2] > 0) and (x2_hom[2] > 0):
                    num_valid += 1
                    X_valid.append(X_list[k])

            # if all points are in front of both cameras, save the camera pair and their corresponding 3D points
            if num_valid == len(X_list):
                valid_poses.append((i, j))
                valid_X.append(X_valid)

    # if there is only one valid pair of cameras, return the corresponding camera pose and the best 3D point estimate
    if len(valid_poses) == 1:
        i, j = valid_poses[0]
        C = (C_list[i] + C_list[j]) / 2
        R = np.dot(R_list[i], R_list[j].T)
        X = np.mean(valid_X[0], axis=0)
        return C, R, X
    else:
        return None


In [ ]:
import numpy as np
import cv2
import scipy.optimize as optimize


def reprojection_loss(X, pts1, pts2, projection_matrix1, projection_matrix2):
    
    p1_1T, p1_2T, p1_3T = projection_matrix1 # rows of projection_matrix1
    p1_1T, p1_2T, p1_3T = p1_1T.reshape(1,-1), p1_2T.reshape(1,-1),p1_3T.reshape(1,-1)

    p2_1T, p2_2T, p2_3T = projection_matrix2 # rows of projection_matrix2
    p2_1T, p2_2T, p2_3T = p2_1T.reshape(1,-1), p2_2T.reshape(1,-1), p2_3T.reshape(1,-1)

    ## reprojection error for reference camera points - j = 1
    u1,v1 = pts1[0], pts1[1]
    u1_proj = np.divide(p1_1T.dot(X) , p1_3T.dot(X))
    v1_proj =  np.divide(p1_2T.dot(X) , p1_3T.dot(X))
    E1 = np.square(v1 - v1_proj) + np.square(u1 - u1_proj)

    
    ## reprojection error for second camera points - j = 2    
    u2,v2 = pts2[0], pts2[1]
    u2_proj = np.divide(p2_1T.dot(X) , p2_3T.dot(X))
    v2_proj =  np.divide(p2_2T.dot(X) , p2_3T.dot(X))    
    E2 = np.square(v2 - v2_proj) + np.square(u2 - u2_proj)
    
    error = E1 + E2
    return error.squeeze()   
    
def non_linear_triangulation(K, x1, x2, X_init_3d, R1, C1, R2, C2):
    
    P1 = get_projection_matrix(R1, C1, K) 
    P2 = get_projection_matrix(R1, C1, K)
    
    assert x1.shape[0] == x2.shape[0] == X_init_3d.shape[0], 'Different shape between 2D to 3d correspondences'

    X_optim_3d = []
    for i in range(len(X_init_3d)):
        optimized_params = optimize.least_squares(fun=reprojection_loss, x0=X_init_3d[i], method="trf", args=[x1[i], x2[i], P1, P2])
        X = optimized_params.x
        X_optim_3d.append(X)
        
    return np.array(X_optim_3d)


In [ ]:
def PnP_reprojection_error(x_3d, pts, K, R, C):
    P = get_projection_matrix(R,C,K)

    E = []
    for X, pt in zip(x_3d, pts):
        p_1T, p_2T, p_3T = P# rows of P
        p_1T, p_2T, p_3T = p_1T.reshape(1,-1), p_2T.reshape(1,-1), p_3T.reshape(1,-1)
        X = h_transform(X.reshape(1,-1)).reshape(-1,1) # make X it a column of homogenous vector
        ## reprojection error for reference camera points 
        u, v = pt[0], pt[1]
        u_proj = np.divide(p_1T.dot(X) , p_3T.dot(X))
        v_proj =  np.divide(p_2T.dot(X) , p_3T.dot(X))

        e = np.square(v - v_proj) + np.square(u - u_proj)

        E.append(e)

    mean_err = np.mean(np.array(E).squeeze())
    return mean_err


def LinearPnP(X, x, K):
    N = X.shape[0]
    
    X = h_transform(X)
    x = h_transform(x)
    
    # normalize x
    K_inv = np.linalg.inv(K)
    x_n = K_inv.dot(x.T).T
    
    for i in range(N):
        X = X[i].reshape((1, 4))
        zeros = np.zeros((1, 4))
        
        u, v, _ = x_n[i]
        
        u_cross = np.array([[0, -1, v],
                            [1,  0 , -u],
                            [-v, u, 0]])
        X_tilde = np.vstack((np.hstack((   X, zeros, zeros)), 
                            np.hstack((zeros,     X, zeros)), 
                            np.hstack((zeros, zeros,     X))))
        a = u_cross.dot(X_tilde)
        
        if i > 0:
            A = np.vstack((A, a))
        else:
            A = a
            
    _, _, V = np.linalg.svd(A)
    P = V[-1].reshape((3, 4))
    R = P[:, :3]
    U_r, D, V_r = np.linalg.svd(R) # to enforce Orthonormality
    R = U_r.dot(V_r)
    
    C = P[:, 3]
    C = - np.linalg.inv(R).dot(C)
    
    if np.linalg.det(R) < 0:
        R = -R
        C = -C
        
    return R, C

In [ ]:
def PnPError(x, X, R, C, K):
    u,v = x
    X = h_transform(X.reshape(1,-1)).reshape(-1,1) # make X it a column of homogenous vector
    C = C.reshape(-1, 1)
    P = get_projection_matrix(R,C,K)
    p1, p2, p3 = P
        
    u_proj = np.divide(p1.dot(X) , p3.dot(X))
    v_proj =  np.divide(p2.dot(X) , p3.dot(X))

    x_proj = np.hstack((u_proj, v_proj))
    x = np.hstack((u, v))
    e = np.linalg.norm(x - x_proj)
    return  e

def PnPRANSAC(K, features, x_3d, n_iterations = 1000, error_thresh = 5):

    inliers_thresh = 0
    # chosen_indices = []
    best_R, best_t = None, None
    n_rows = x_3d.shape[0]
    
    for i in range(0, n_iterations):
        
        #select 6 points randomly
        random_indices = np.random.choice(n_rows, size=6)
        X, x = x_3d[random_indices], features[random_indices]
        
        R,C = LinearPnP(X, x, K)
        
        indices = []
        if R is not None:
            for j in range(n_rows):
                feature = features[j]
                X = x_3d[j]
                error = PnPError(feature, X, R, C, K)

                if error < error_thresh:
                    indices.append(j)
                    
        if len(indices) > inliers_thresh:
            inliers_thresh = len(indices)
            chosen_indices = indices
            best_R = R
            best_t = C
            
    return best_R, best_t

In [ ]:
def PnPLoss(X0, x3D, pts, K):
    
    Q, C = X0[:4], X0[4:].reshape(-1,1)
    R = get_rotation(Q)
    P = get_projection_matrix(R,C,K)
    
    E = []
    for X, pt in zip(x3D, pts):

        p_1T, p_2T, p_3T = P# rows of P
        p_1T, p_2T, p_3T = p_1T.reshape(1,-1), p_2T.reshape(1,-1), p_3T.reshape(1,-1)


        X = h_transform(X.reshape(1,-1)).reshape(-1,1) 
        ## reprojection error for reference camera points 
        u, v = pt[0], pt[1]
        u_proj = np.divide(p_1T.dot(X) , p_3T.dot(X))
        v_proj =  np.divide(p_2T.dot(X) , p_3T.dot(X))

        e = np.square(v - v_proj) + np.square(u - u_proj)

        E.append(e)

    sumError = np.mean(np.array(E).squeeze())
    return sumError

def NonLinearPnP(K, pts, x3D, R0, C0):
    """    
    K : Camera Matrix
    pts1, pts2 : 2D point correspondences
    x_init_3d :  initial 3D point 
    R2, C2 : relative camera pose - estimated from PnP
    Returns:
        x_optim_3d : optimized 3D points
    """

    Q = get_quaternion(R0)
    X0 = [Q[0] ,Q[1],Q[2],Q[3], C0[0], C0[1], C0[2]] 

    optimized_params = optimize.least_squares(
        fun = PnPLoss,
        x0=X0,
        method="trf",
        args=[x3D, pts, K])
    X1 = optimized_params.x
    Q = X1[:4]
    C = X1[4:]
    R = get_rotation(Q)
    return R, C

In [ ]:
def get_obs_index_and_viz_mat(X_found, filtered_feature_flag, nCam):
    # find the 3d points such that they are visible in either of the cameras < nCam
    bin_temp = np.zeros((filtered_feature_flag.shape[0]), dtype = int)
    for n in range(nCam + 1):
        bin_temp = bin_temp | filtered_feature_flag[:,n]

    X_idx = np.where((X_found.reshape(-1)) & (bin_temp))
    
    visiblity_matrix = X_found[X_idx].reshape(-1,1)
    for n in range(nCam + 1):
        visiblity_matrix = np.hstack((visiblity_matrix, filtered_feature_flag[X_idx, n].reshape(-1,1)))

    _, c = visiblity_matrix.shape
    return X_idx, visiblity_matrix[:, 1:c]

In [ ]:
def get_2d_pts(X_idx, visiblity_matrix, feature_x, feature_y):
    "Get 2D Points from the feature x and feature y having same index from the visibility matrix"
    pts_2d = []
    visible_feature_x = feature_x[X_idx]
    visible_feature_y = feature_y[X_idx]
    h, w = visiblity_matrix.shape
    for i in range(h):
        for j in range(w):
            if visiblity_matrix[i,j] == 1:
                pt = np.hstack((visible_feature_x[i,j], visible_feature_y[i,j]))
                pts_2d.append(pt)
    return np.array(pts_2d).reshape(-1, 2) 

def get_cam_pt_indices(visiblity_matrix):

    "From Visibility Matrix take away indices of point visible from Camera pose by taking indices of cam as well"

    cam_indices = []
    pt_indices = []
    h, w = visiblity_matrix.shape
    for i in range(h):
        for j in range(w):
            if visiblity_matrix[i,j] == 1:
                cam_indices.append(j)
                pt_indices.append(i)

    return np.array(cam_indices).reshape(-1), np.array(pt_indices).reshape(-1)

def bundle_adjustment_sparsity(X_found, filtered_feature_flag, nCam):
    
    "Create Sparsity Matrix"

    number_of_cam = nCam + 1
    X_idx, visiblity_matrix = get_obs_index_and_viz_mat(X_found.reshape(-1), filtered_feature_flag, nCam)
    n_observations = np.sum(visiblity_matrix)
    n_points = len(X_idx[0])

    m = n_observations * 2
    n = number_of_cam * 6 + n_points * 3   #Here we don't take focal length and 2 radial distortion parameters, i.e no - 6. We just refine orientation and translation of 3d point and not cam parameters.
    A = lil_matrix((m, n), dtype=int)
    # print(m, n)


    i = np.arange(n_observations)
    camera_indices, point_indices = get_cam_pt_indices(visiblity_matrix)

    for s in range(6):
        A[2 * i, camera_indices * 6 + s] = 1
        A[2 * i + 1, camera_indices * 6 + s] = 1

    for s in range(3):
        A[2 * i, (nCam)* 6 + point_indices * 3 + s] = 1
        A[2 * i + 1, (nCam) * 6 + point_indices * 3 + s] = 1

    return A    

def project(points_3d, camera_params, K):
    def project_pt_(R, C, pt_3d, K):
        P2 = np.dot(K, np.dot(R, np.hstack((np.identity(3), -C.reshape(3,1)))))
        x_3d_4 = np.hstack((pt_3d, 1))
        x_proj = np.dot(P2, x_3d_4.T)
        x_proj /= x_proj[-1]
        return x_proj

    x_proj = []
    for i in range(len(camera_params)):
        R = get_rotation(camera_params[i, :3], 'e')
        C = camera_params[i, 3:].reshape(3,1)
        pt_3d = points_3d[i]
        pt_proj = project_pt_(R, C, pt_3d, K)[:2]
        x_proj.append(pt_proj)    
    return np.array(x_proj)

def rotate(points, rot_vecs):
    """Rotate points by given rotation vectors.
    
    Rodrigues' rotation formula is used.
    """
    theta = np.linalg.norm(rot_vecs, axis=1)[:, np.newaxis]
    with np.errstate(invalid='ignore'):
        v = rot_vecs / theta
        v = np.nan_to_num(v)
    dot = np.sum(points * v, axis=1)[:, np.newaxis]
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)

    return cos_theta * points + sin_theta * np.cross(v, points) + dot * (1 - cos_theta) * v

def fun(x0, nCam, n_points, camera_indices, point_indices, points_2d, K):
    """Compute residuals.
    
    `params` contains camera parameters and 3-D coordinates.
    """
    number_of_cam = nCam + 1
    camera_params = x0[:number_of_cam * 6].reshape((number_of_cam, 6))
    points_3d = x0[number_of_cam * 6:].reshape((n_points, 3))
    points_proj = project(points_3d[point_indices], camera_params[camera_indices], K)
    error_vec = (points_proj - points_2d).ravel()
    
    return error_vec  


def bundle_adjustment(X_idx,visibility_matrix,X_all,X_found,feature_x,feature_y, filtered_feature_flag, R_set, C_set, K, nCam):
    points_3d = X_all[X_idx]
    points_2d = get_2d_pts(X_idx,visibility_matrix,feature_x,feature_y)

    RC = []
    for i in range(nCam+1):
        C, R = C_set[i], R_set[i]
        Q = get_euler(R)
        RC_ = [Q[0], Q[1], Q[2], C[0], C[1], C[2]]
        RC.append(RC_)
    RC = np.array(RC, dtype=object).reshape(-1,6)

    x0 = np.hstack((RC.ravel(), points_3d.ravel()))
    n_pts = points_3d.shape[0]

    camera_indices, points_indices = get_cam_pt_indices(visibility_matrix)

    A = bundle_adjustment_sparsity(X_found,filtered_feature_flag,nCam)
    t0 = time.time()
    res = least_squares(fun,x0,jac_sparsity=A, verbose=2,x_scale='jac', ftol=1e-10, method='trf',
                        args=(nCam, n_pts, camera_indices, points_indices, points_2d,K))

    t1 = time.time()
    print("Time required to run Bundle Adj: ", t1-t0, "s \nA matrix shape: ",A.shape,"\n######")

    x1 = res.x
    no_of_cams = nCam + 1
    cam_param_optim = x1[:no_of_cams*6].reshape((no_of_cams,6))
    pts_3d_optim = x1[no_of_cams*6:].reshape((n_pts,3))

    X_all_optim = np.zeros_like(X_all)
    X_all_optim[X_idx] = pts_3d_optim

    C_optim , R_optim = [], []
    for i in range(len(cam_param_optim)):
        R = get_rotation(cam_param_optim[i,:3], 'e')
        C = cam_param_optim[i,3:].reshape(3,1)
        C_optim.append(C)
        R_optim.append(R)

    return R_optim, C_optim, X_all_optim